In [327]:
import mysql.connector
import pandas as pd

In [329]:
account = pd.read_excel("./dataset/SQL Test Data.xlsx", sheet_name="Account")
betting = pd.read_excel("./dataset/SQL Test Data.xlsx", sheet_name="Betting")
customer = pd.read_excel("./dataset/SQL Test Data.xlsx", sheet_name="Customer")
product = pd.read_excel("./dataset/SQL Test Data.xlsx", sheet_name="Product")

student = pd.read_excel("./dataset/SQL Test Data.xlsx", sheet_name="Student", header=1)
student = student.drop(index=0) #Drop second row due to data not there, continue on the 3rd row for data insertion

school = pd.read_excel("./dataset/SQL Test Data.xlsx", sheet_name="School", header=1)
school = school.drop(index=0) #Drop second row due to data not there, continue on the 3rd row for data insertion

In [330]:
gamble_db = mysql.connector.connect(
    host="localhost",      
    port=3306,             
    user="root",
    password="SQLNikki272829",
    database="IronHackGamble"
)

print(f"There is a confirmed connection {gamble_db}")  

There is a confirmed connection <mysql.connector.connection.MySQLConnection object at 0x0000019CBBDA6810>


In [331]:
mycursor = gamble_db.cursor()

mycursor.execute("CREATE DATABASE IF NOT EXISTS IronHackGamble")

In [333]:
mycursor.execute("SHOW DATABASES")

for x in mycursor:
  print(x)

('bank',)
('information_schema',)
('ironhackgamble',)
('lab_mysql',)
('mysql',)
('performance_schema',)
('publications',)
('sakila',)
('sys',)


In [334]:
mycursor.execute("USE IronHackGamble")

In [335]:
#Account table query
mycursor = gamble_db.cursor()

account_table_query = """
CREATE TABLE IF NOT EXISTS Account (
  account_num VARCHAR(7) PRIMARY KEY,
  customer_id INT,
  account_location CHAR(3),
  currency_code CHAR(3),
  daily_deposit_limit DECIMAL(10, 2),
  stake_scale DECIMAL(5, 2),
  source_prod VARCHAR(2)
);
"""

mycursor.execute(account_table_query)

In [336]:
#Betting table query
mycursor = gamble_db.cursor()

betting_table_query = """
CREATE TABLE IF NOT EXISTS Betting (
  account_num VARCHAR(7),
  bet_date TIMESTAMP,
  class_id VARCHAR(20),
  category_id INT,
  source CHAR(1),
  bet_count INT,
  bet_amount DECIMAL(10, 2),
  win_amount DECIMAL(10, 2),
  product VARCHAR(20),
  FOREIGN KEY (account_num) REFERENCES Account(account_num)
);
"""
mycursor.execute(betting_table_query)

In [337]:
#Customer table query
mycursor = gamble_db.cursor()

customer_table_query = """
CREATE TABLE IF NOT EXISTS Customer (
  cust_id INT PRIMARY KEY,
  account_location CHAR(3),
  title VARCHAR(10),
  first_name VARCHAR(50),
  last_name VARCHAR(50),
  create_date DATE,
  country_code CHAR(2),
  language CHAR(2),
  status CHAR(1),
  date_of_birth DATE,
  contact CHAR(1),
  customer_group VARCHAR(20)
);
"""
mycursor.execute(customer_table_query)

In [338]:
#Product table query
mycursor = gamble_db.cursor()

product_table_query = """
CREATE TABLE IF NOT EXISTS Product (
  CLASSID VARCHAR(20),
  CATEGORYID INT,
  product VARCHAR(50),
  sub_product VARCHAR(50),
  description VARCHAR(100),
  bet_or_play INT
);
"""
mycursor.execute(product_table_query)

In [339]:
mycursor.execute("SHOW TABLES")

for x in mycursor:
  print(x)

('account',)
('betting',)
('customer',)
('product',)
('school',)
('student',)


In [342]:
#Insert data for account table
for index,row in account.iterrows():
    account_num = row['AccountNo']
    customer_id = row['CustId']
    account_location = row['AccountLocation']
    currency_code = row['CurrencyCode']
    daily_deposit_limit = row['DailyDepositLimit'] 
    stake_scale = row['StakeScale']
    source_prod = row['SourceProd']
    
    account_insert_query = """
        INSERT INTO Account (account_num, customer_id, account_location, currency_code, daily_deposit_limit, stake_scale, source_prod)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE
        customer_id = VALUES(customer_id), 
        account_location = VALUES(account_location),
        currency_code = VALUES(currency_code),
        daily_deposit_limit = VALUES(daily_deposit_limit),
        stake_scale = VALUES(stake_scale),
        source_prod = VALUES(source_prod);
        """
    
    
    mycursor.execute(account_insert_query, (account_num, customer_id, account_location, currency_code, daily_deposit_limit, stake_scale, source_prod))

In [350]:
#Insert data for betting table
for index, row in betting.iterrows():
    account_num = row['AccountNo']
    bet_date = row['BetDate']
    class_id = row['ClassId']
    category_id = row['CategoryId']
    source = row['Source']
    bet_count = row['BetCount']
    bet_amount = row['Bet_Amt']
    win_amount = row['Win_Amt']
    product = row['Product']

    bet_date = pd.to_datetime(row['BetDate']).strftime('%Y-%m-%d %H:%M:%S') if not pd.isna(row['BetDate']) else None
    
    betting_insert_query = """
        INSERT INTO Betting (account_num, bet_date, class_id, category_id, source, bet_count, bet_amount, win_amount, product)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
    
    mycursor.execute(betting_insert_query, (account_num, bet_date, class_id, category_id, source, bet_count, bet_amount, win_amount, product))

In [352]:
#Insert data for the customer table
for index, row in customer.iterrows():
    cust_id = row['CustId']
    account_location = row['AccountLocation']
    title = row['Title']
    first_name = row['FirstName']
    last_name = row['LastName']
    create_date = row['CreateDate']
    country_code = row['CountryCode']
    language = row['Language']
    status = row['Status']
    date_of_birth = row['DateOfBirth']
    contact = row['Contact']
    customer_group = row['CustomerGroup']

    create_date = pd.to_datetime(row['CreateDate']).strftime('%Y-%m-%d %H:%M:%S') if not pd.isna(row['CreateDate']) else None
    date_of_birth = pd.to_datetime(row['DateOfBirth']).strftime('%Y-%m-%d %H:%M:%S') if not pd.isna(row['DateOfBirth']) else None

    
    customer_insert_query = """
        INSERT INTO Customer (cust_id, account_location, title, first_name, last_name, create_date, country_code, language, status, date_of_birth, contact, customer_group)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
    
    mycursor.execute(customer_insert_query, (cust_id, account_location, title, first_name, last_name, create_date, country_code, language, status, date_of_birth, contact, customer_group))

IntegrityError: 1062 (23000): Duplicate entry '4188499' for key 'customer.PRIMARY'

In [ ]:
#Insert data for the product table
for index, row in product.iterrows():
    CLASSID = row['CLASSID']
    CATEGORYID = row['CATEGORYID']
    product_name = row['product']
    sub_product = row['sub_product']
    description = row['description']
    bet_or_play = row['bet_or_play']
    
    product_insert_query = """
        INSERT INTO Product (CLASSID, CATEGORYID, product, sub_product, description, bet_or_play)
        VALUES (%s, %s, %s, %s, %s, %s)
        """
    
    mycursor.execute(product_insert_query, (CLASSID, CATEGORYID, product_name, sub_product, description, bet_or_play))

In [ ]:
#Create Students table
mycursor = gamble_db.cursor()

student_table_query = """
CREATE TABLE IF NOT EXISTS Student (
  student_id INT(4) PRIMARY KEY,      
  student_name VARCHAR(50) NOT NULL,  
  city VARCHAR(50) NOT NULL,         
  school_id INT NOT NULL,             
  GPA DECIMAL(3,2) NOT NULL           
);
"""
mycursor.execute(student_table_query)

In [ ]:
#Insert Student data
for index, row in student_school.iterrows():
    student_id = row['student_id']
    student_name = row['student_name']
    city = row['city']
    school_id = row['school_id']
    GPA = row['GPA']
    
    student_insert_query = """
        INSERT INTO Student (student_id, student_name, city, school_id, GPA)
        VALUES (%s, %s, %s, %s, %s)
        """
    
    mycursor.execute(student_insert_query, (student_id, student_name, city, school_id, GPA))

In [360]:
#Create School table
mycursor = gamble_db.cursor()

school_table_query = """
CREATE TABLE IF NOT EXISTS School (
  school_id INT PRIMARY KEY,        
  school_name VARCHAR(100) NOT NULL,
  city VARCHAR(50) NOT NULL            
);
"""
mycursor.execute(school_table_query)

In [362]:
#Insert School data
for index, row in student_school.iterrows():
    school_id = row['school_id']
    school_name = row['school_name']
    city = row['city']
    
    school_insert_query = """
        INSERT INTO School (school_id, school_name, city)
        VALUES (%s, %s, %s)
        ON DUPLICATE KEY UPDATE 
            school_name = VALUES(school_name),
            city = VALUES(city);
    """
    
    mycursor.execute(school_insert_query, (school_id, school_name, city))

DatabaseError: 1205 (HY000): Lock wait timeout exceeded; try restarting transaction

In [ ]:
# Commit the changes
gamble_db.commit()

# Close the cursor and connection
mycursor.close()
gamble_db.close()